In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

import gym
import trading_env

In [2]:
env = gym.make('trading-v0')

In [8]:
print(env.step(None))

(array([[  854.66      ,   391.50560803],
       [  853.        ,   660.2372293 ],
       [  853.06      ,   674.91328127],
       [  853.39      ,   353.05242027],
       [  853.07      ,   335.0114585 ],
       [  851.6       ,  1312.26131638],
       [  852.65      ,   639.85362155],
       [  853.63      ,   585.62493721],
       [  854.31      ,   260.13585904],
       [  854.18      ,   329.97176459],
       [  853.93      ,   252.59454638],
       [  853.42      ,   305.44398991],
       [  852.45      ,   378.30333085],
       [  852.04      ,   295.21084624],
       [  848.83      ,  1002.92702776],
       [  845.01      ,  1060.81713567],
       [  841.51      ,  1580.08971367],
       [  841.28      ,   864.54532547],
       [  841.21      ,   732.52409934],
       [  839.1       ,  1278.66365683],
       [  838.95      ,   338.7200612 ],
       [  840.        ,   674.4560149 ],
       [  842.75      ,   549.47246754],
       [  843.74      ,   504.60043757],
       [  842.4

In [4]:
df_year = pd.read_pickle("data/ETH_USD_2017_2018_15min.pkl")
display(df_year.head(5))

,time,low,high,open,close,volume
0,2017-01-01 00:00:00,8.11,8.16,8.16,8.16,378.420945
1,2017-01-01 00:15:00,8.16,8.24,8.16,8.24,252.619480
2,2017-01-01 00:30:00,8.24,8.27,8.24,8.27,237.637246
3,2017-01-01 00:45:00,8.25,8.27,8.27,8.26,109.079011
4,2017-01-01 01:00:00,8.26,8.29,8.26,8.29,270.898827


In [ ]:
n_features = 4 # Low, High Close, Volume
n_days_window = 30
n_window = (n_days_window * 24 * 60) // 15

In [ ]:
a = 1+1
print(a)